# Gunter's Space Page GSP Scraper

In [7]:
import bs4
import requests
import pandas as pd
import os

In [12]:
#url = "https://space.skyrocket.de/doc_sdat/globalstar.htm"
oneweblist = [
    "https://space.skyrocket.de/doc_sdat/oneweb.htm"
]

starlinklist = [
    "https://space.skyrocket.de/doc_sdat/starlink-v0-9.htm",
    "https://space.skyrocket.de/doc_sdat/starlink-v1-0.htm",
    "https://space.skyrocket.de/doc_sdat/starlink-v1-5.htm",
    "https://space.skyrocket.de/doc_sdat/starlink-v2-mini.htm"
]

globalstarlist = [
    "https://space.skyrocket.de/doc_sdat/globalstar.htm",
    "https://space.skyrocket.de/doc_sdat/globalstar-2.htm"
]

iridiumlist = [
    "https://space.skyrocket.de/doc_sdat/iridium.htm",
    "https://space.skyrocket.de/doc_sdat/iridium-next.htm"
]

orbcommlist = [
    "https://space.skyrocket.de/doc_sdat/orbcomm.htm",
    "https://space.skyrocket.de/doc_sdat/orbcomm-ql.htm",
    "https://space.skyrocket.de/doc_sdat/orbcomm-2.htm"
]

swarmlist = [
    "https://space.skyrocket.de/doc_sdat/spacebee.htm",
    "https://space.skyrocket.de/doc_sdat/spacebee-5.htm",
    "https://space.skyrocket.de/doc_sdat/spacebee-10.htm"
]

In [15]:
def scrape_url(url):
    """
    
    """
    response = requests.get(url)
    soup = bs4.BeautifulSoup(response.text)

    satdata = soup.find("table", {"id": "satdata"})
    satlist = soup.find("table", {"id": "satlist"})

    def parse_row(trrow):
        tdlist = trrow.find_all("td")
        namelist = [td.text for td in tdlist]
        return namelist

    satlist_dict = {
        "Satellite": [],
        "COSPAR": [],
        "Date": [],
        "LS": [],
        "Failed": [],
        "Launche Vehicle": [],
        "Remarks": []
    }

    for trrow in satlist.find_all("tr")[1:]:
        #if
        namelist = parse_row(trrow)
        datestr = namelist[2].lower()
        if(datestr == "cancelled" or datestr == "2023" or datestr == "not launched"):
            continue
        try:
            date = pd.to_datetime(datestr, format="%d.%m.%Y")
        except:
            continue
        satlist_dict["Satellite"].append(namelist[0])
        satlist_dict["COSPAR"].append(namelist[1])
        satlist_dict["Date"].append(date)
        satlist_dict["LS"].append(namelist[3])
        satlist_dict["Failed"].append(namelist[4])
        satlist_dict["Launche Vehicle"].append(namelist[5])
        satlist_dict["Remarks"].append(namelist[6])

    df = pd.DataFrame(satlist_dict)
    return df

def scrape_urllist(urllist):
    """
    Takes a list of Gunter's Space Page urls to scrape from
    
    Returns dataframe of all the satellites found
    """
    dflist = [scrape_url(url) for url in starlinklist]
    return pd.concat(dflist)

## Starlink Specific Stuff

In [16]:
df = scrape_urllist(starlinklist)
df["Revision"] = df["Satellite"].apply(lambda sat: sat.split(" ")[1])

In [52]:
launch_dates_df = pd.DataFrame(list(set([rd for rd in zip(df["Revision"], df["Date"])])))
launch_dates_df.to_csv("data/starlink_launchdates.csv")